Climate pulse SST analysis with Matrix Profile (MP) and Interval Matrix Profile (IMP)
==============

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import sys
sys.path.insert(0,'../build/src/')
import libimp
plotly.offline.init_notebook_mode()

ModuleNotFoundError: No module named 'libimp'

In [ ]:
!pwd

In [ ]:
!ls ../build/src

## Sea Surface Temperature

In [ ]:
data_path = "../Data/climate_pulse/SST/"
df = pd.read_csv(data_path+"2024_era5_daily_series_sst_60S-60N_ocean.csv",skiprows=18)
df = df[df["status"]=="FINAL"]
ts_sst = df["sst"].values.astype(np.float64)
date_sst = df["date"].values.astype(np.datetime64)
window_size = 7
interval_length = 61
exclude = 7
exclude_diag = False
start_year = np.datetime64(date_sst[0], 'Y').astype(int) + 1970
end_year = np.datetime64(date_sst[-1], 'Y').astype(int) + 1970
period_starts = []
for year in range(start_year, end_year+1):
    period_starts.append(np.where(date_sst == np.datetime64(str(year) + "-01-01"))[0][0])
block_width = 500
block_height = 500

In [ ]:
start_year, end_year

In [ ]:
plt.figure(figsize=(13,6))
plt.plot(date_sst, ts_sst)
plt.show()

In [ ]:
mp_sst, mp_index_sst = libimp.STOMP_double(ts_sst, window_size, exclude, block_width, block_width)
imp_sst, imp_index_sst = libimp.imp_STOMP_double(ts_sst, window_size, period_starts, interval_length, exclude)

imp_3NN, imp_3NN_index = libimp.BIMP_kNN_double(ts_sst, window_size, period_starts, interval_length, exclude, 3, exclude_diag)
imp_5NN, imp_5NN_index = libimp.BIMP_kNN_double(ts_sst, window_size, period_starts, interval_length, exclude, 5, exclude_diag)
imp_10NN, imp_10NN_index = libimp.BIMP_kNN_double(ts_sst, window_size, period_starts, interval_length, exclude, 10, exclude_diag)



size = len(mp_sst)

In [ ]:
plt.plot(mp_sst)

In [ ]:
type(mp_sst), type(mp_index_sst)

In [ ]:
type(imp_sst), type(imp_index_sst)

In [ ]:
mp_date = date_sst[:size]
ind_discord_mp = np.argpartition(mp_sst, -1)[-1:]
ind_discord_imp = np.argpartition(imp_sst, -30)[-30:]

cut_off_mp = np.min(mp_sst[ind_discord_mp])
cut_off_imp = np.min(imp_sst[ind_discord_imp])

hover_text_mp = [f"NN: {date_sst[ind]}" for ind in mp_index_sst]
hover_text_imp = [f"NN: {date_sst[ind]}" for ind in imp_index_sst]
fig = go.Figure()
# Add traces
fig.add_trace(go.Scatter(x=mp_date, y=imp_sst, mode='lines', name='IMP', text=hover_text_imp))
fig.add_trace(go.Scatter(x=mp_date, y=mp_sst, mode='lines', name='MP', text=hover_text_mp))
fig.add_trace(go.Scatter(x=mp_date, y=np.zeros(mp_date.shape[0]) + cut_off_mp, mode='lines', line=dict(dash='dash',color='red'), name='MP cutoff', hoverinfo='skip'))
fig.add_trace(go.Scatter(x=mp_date, y=np.zeros(mp_date.shape[0]) + cut_off_imp, mode='lines', line=dict(dash='dash',color='blue'), name='IMP cutoff', hoverinfo='skip'))
# Update layout
fig.update_layout(
    title="Sample Plot",
    xaxis_title="Date",
    yaxis_title="Distance",
    legend_title="Method",
    autosize=True,
    #width=1300,
    height=800,
    hovermode='x unified',
)
# Show the figure
fig.show()

In [ ]:
mp_date = date_sst[:size]
ind_discord_3nn = np.argpartition(imp_3NN, -30)[-30:]
ind_discord_5nn = np.argpartition(imp_5NN, -30)[-30:]
ind_discord_10nn = np.argpartition(imp_10NN, -30)[-30:]

cut_off_3 = np.min(imp_3NN[ind_discord_3nn])
cut_off_5 = np.min(imp_5NN[ind_discord_5nn])
cut_off_10 = np.min(imp_10NN[ind_discord_10nn])

hover_text_3 = [f"NN: {mp_date[ind]}" for ind in imp_3NN_index]
hover_text_5 = [f"NN: {mp_date[ind]}" for ind in imp_5NN_index]
hover_text_10 = [f"NN: {mp_date[ind]}" for ind in imp_10NN_index]
fig = go.Figure()
# Add traces
fig.add_trace(go.Scatter(x=mp_date, y=imp_10NN, mode='lines', name='10', text=hover_text_10))
fig.add_trace(go.Scatter(x=mp_date, y=imp_5NN, mode='lines', name='5', text=hover_text_5))
fig.add_trace(go.Scatter(x=mp_date, y=imp_3NN, mode='lines', name='3', text=hover_text_3))
fig.add_trace(go.Scatter(x=mp_date, y=imp_sst, mode='lines',line=dict(color='blue'), name='IMP', text=hover_text_imp))
fig.add_trace(go.Scatter(x=mp_date, y=mp_sst, mode='lines',line=dict(color='yellow'), name='MP', text=hover_text_mp))

fig.add_trace(go.Scatter(x=mp_date, y=np.zeros(mp_date.shape[0]) + cut_off_3, mode='lines', line=dict(dash='dash',color='red'), name='3 cutoff', hoverinfo='skip'))
fig.add_trace(go.Scatter(x=mp_date, y=np.zeros(mp_date.shape[0]) + cut_off_5, mode='lines', line=dict(dash='dash',color='blue'), name='5 cutoff', hoverinfo='skip'))
fig.add_trace(go.Scatter(x=mp_date, y=np.zeros(mp_date.shape[0]) + cut_off_10, mode='lines', line=dict(dash='dash',color='blue'), name='10 cutoff', hoverinfo='skip'))
fig.add_trace(go.Scatter(x=mp_date, y=np.zeros(mp_date.shape[0]) + cut_off_imp, mode='lines', line=dict(dash='dash',color='blue'), name='1NN cutoff', hoverinfo='skip'))

# Update layout
fig.update_layout(
    title="Sample Plot",
    xaxis_title="Date",
    yaxis_title="Distance",
    legend_title="Method",
    autosize=True,
    #width=1300,
    height=800,
    hovermode='x unified',
)
# Show the figure
fig.show()

In [ ]:
date = np.where(mp_date==np.datetime64("2023-01-01"))[0][0]
mp_dates = date_sst[date:size]
lw=2
fig = go.Figure()
# Add traces
fig.add_trace(go.Scatter(x=mp_dates, y=np.zeros(len(mp_dates)) + cut_off_mp, mode='lines', line=dict(dash='dash',color='black'), name='Top 1 discord (MP)'))
fig.add_trace(go.Scatter(x=mp_dates, y=imp_10NN[date:], mode='lines', line=dict(color='purple', width=lw), name='c10NN IMP', text=hover_text_10))
fig.add_trace(go.Scatter(x=mp_dates, y=imp_5NN[date:],  mode='lines', line=dict(color='orange', width=lw), name='c5NN IMP', text=hover_text_5))
fig.add_trace(go.Scatter(x=mp_dates, y=imp_3NN[date:],  mode='lines', line=dict(color='green', width=lw), name='c3NN IMP', text=hover_text_3))
fig.add_trace(go.Scatter(x=mp_dates, y=imp_sst[date:],  mode='lines', line=dict(color='red', width=lw), name='IMP', text=hover_text_imp))
fig.add_trace(go.Scatter(x=mp_dates, y=mp_sst[date:],   mode='lines', line=dict(color='blue', width=lw), name='MP', text=hover_text_mp))

# Update layout
fig.update_layout(
    template="plotly_white",
    xaxis_title="Date",
    yaxis_title="Distance",
    autosize=True,
    width=1200,
    height=900,
    hovermode='x unified',
    font=dict(
        family="Latin Modern Roman",
        size=30,  
        color="Black"
    )
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.05,
    bgcolor='rgba(0, 0, 0, 0)'),
     yaxis=dict(
        tickvals=[0,0.25,0.5,0.75,1], 
        ticktext=[0,0.25,0.5,0.75,1], 
        showgrid=True
    )
                 )
fig.update_layout(margin=dict(l=20, r=0, t=0, b=20))

fig.write_image("2024_sst_imp.pdf", engine="kaleido")
fig.show()

In [ ]:
fig = px.scatter(x=[0, 1, 2, 3, 4], y=[0, 1, 4, 9, 16])
fig.show()
fig.write_image("random.pdf")

In [ ]:
fig = go.Figure()

# Plot historical data from previous years
for year in range(start_year, end_year-1):
    dates = np.where((date_sst >= np.datetime64(f"{year}-01-01")) & (date_sst <= np.datetime64(f"{year}-12-31")))[0]
    x = np.arange(0, 365, 1)
    fig.add_trace(go.Scatter(x=x, y=ts_sst[dates], mode='lines', line=dict(color="gray", width=1), showlegend=False))

# Plot 2023 data
dates = np.where((date_sst >= np.datetime64(f"{2023}-01-01")) & (date_sst <= np.datetime64(f"{2023}-12-31")))[0]
x = np.arange(0, 365, 1)
fig.add_trace(go.Scatter(x=x, y=ts_sst[dates], mode='lines', line=dict(color="red", width=2), name="2023"))

# Plot 2024 data
dates = np.where((date_sst >= np.datetime64(f"{2024}-01-01")) & (date_sst <= np.datetime64(f"{2024}-12-31")))[0]
x = np.arange(0, 365, 1)
fig.add_trace(go.Scatter(x=x, y=ts_sst[dates], mode='lines', line=dict(color="green", width=2), name="2024"))

# Update layout
fig.update_layout(
    template="plotly_white",
    xaxis_title="Date",
    yaxis_title="Temperature (°C)",
    autosize=True,
    width=1200,
    height=900,
    font=dict(
        family="Latin Modern Roman",
        size=30,  
        color="Black"
    ),
    
)

first_day_of_month_indices = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334]  
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig.update_layout(
    xaxis=dict(
        tickvals=first_day_of_month_indices, 
        ticktext=month_labels,
        showgrid=False
    ),
    yaxis=dict(
        tickvals=[20,20.5,21], 
        ticktext=[20,20.5,21],
        range=[19.57,21.11],
        showgrid=True
    )
)
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.05,
    xanchor="left",
    x=0.05
))
fig.update_layout(margin=dict(l=20, r=25, t=0, b=20))

fig.write_image("2024_sst.pdf", engine="kaleido")
fig.show()

In [ ]:
date_sst

In [ ]:
mp_values, mp_counts = np.unique(mp_sst, return_counts=True)
imp_values, imp_counts = np.unique(imp_sst, return_counts=True)
imp3_values, imp3_counts = np.unique(imp_3NN, return_counts=True)

df_mp = pd.DataFrame({'Distance': mp_values, 'Count': mp_counts, 'Method': 'MP'})
df_imp = pd.DataFrame({'Distance': imp_values, 'Count': imp_counts, 'Method': 'IMP'})
df_imp3 = pd.DataFrame({'Distance': imp3_values, 'Count': imp3_counts, 'Method': 'c3NN IMP'})


# Combine the two DataFrames
df = pd.concat([df_mp, df_imp, df_imp3], ignore_index=True)

fig = px.histogram(df, x="Distance", y="Count", color="Method", marginal="box",
                   hover_data=df.columns)

fig.update_layout(
    template="plotly_white",
    xaxis_title="Distance",
    yaxis_title="Count",
    autosize=True,
    width=800,
    height=500,
    #bargap=0,
    font=dict(
        family="Latin Modern Roman",
        size=20,  
        color="Black"
    ),
)
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig.update_layout(legend=dict(
    yanchor="middle",
    y=0.5,
    xanchor="right",
    x=0.9,
))

fig.show()

In [ ]:
help(libimp)

In [ ]:
interval_length

In [ ]:
k = 10
window_size = 14
left_imp, left_imp_index = libimp.left_BIMP_kNN_double(ts_sst, window_size, period_starts, interval_length, exclude, 1, exclude_diag)
right_imp, right_imp_index = libimp.right_BIMP_kNN_double(ts_sst, window_size, period_starts, interval_length, exclude, 1, exclude_diag)

left_imp_10, left_imp_index_10 = libimp.left_BIMP_kNN_double(ts_sst, window_size, period_starts, interval_length, exclude, k, exclude_diag)
right_imp_10, right_imp_index_10 = libimp.right_BIMP_kNN_double(ts_sst, window_size, period_starts, interval_length, exclude, k, exclude_diag)

In [ ]:
plt.plot(left_imp[5*365:-5*365], label="Left")
plt.plot(right_imp[5*365:-5*365],label="Right")
plt.legend()
plt.show()

In [ ]:
# Create hover text for each trace
hover_text_imp = [f"Date: {mp_date[ind]}, Value: {val}" for ind, val in enumerate(imp_sst[k*365:-k*365])]
hover_text_left = [f"Date: {mp_date[ind]}, Value: {val}" for ind, val in enumerate(left_imp[k*365:-k*365])]
hover_text_right = [f"Date: {mp_date[ind]}, Value: {val}" for ind, val in enumerate(right_imp[k*365:-k*365])]
hover_text_left_10 = [f"Date: {mp_date[ind]}, Value: {val}" for ind, val in enumerate(left_imp_10[k*365:-k*365])]
hover_text_right_10 = [f"Date: {mp_date[ind]}, Value: {val}" for ind, val in enumerate(right_imp_10[k*365:-k*365])]

# Create the figure
fig = go.Figure()

# IMP trace with hover text
fig.add_trace(go.Scatter(
    x=mp_date[k*365:-k*365],         # Date array for x-axis
    y=imp_sst[k*365:-k*365],         # IMP data
    mode='lines',
    name='IMP',
    text=hover_text_imp,              # Custom hover text for IMP
))

# Left trace with hover text
fig.add_trace(go.Scatter(
    x=mp_date[k*365:-k*365],         # Date array for x-axis
    y=left_imp[k*365:-k*365],        # Left data
    mode='lines',
    name='Left',
    text=hover_text_left,             # Custom hover text for Left
))

# Right trace with hover text
fig.add_trace(go.Scatter(
    x=mp_date[k*365:-k*365],         # Date array for x-axis
    y=right_imp[k*365:-k*365],       # Right data
    mode='lines',
    name='Right',
    text=hover_text_right,            # Custom hover text for Right
))

# Left 10 trace with hover text
fig.add_trace(go.Scatter(
    x=mp_date[k*365:-k*365],         # Date array for x-axis
    y=left_imp_10[k*365:-k*365],     # Left 10 data
    mode='lines',
    name='Left 10',
    text=hover_text_left_10,          # Custom hover text for Left 10
))

# Right 10 trace with hover text
fig.add_trace(go.Scatter(
    x=mp_date[k*365:-k*365],         # Date array for x-axis
    y=right_imp_10[k*365:-k*365],    # Right 10 data
    mode='lines',
    name='Right 10',
    text=hover_text_right_10,         # Custom hover text for Right 10
))

# Customize layout
fig.update_layout(
    title='Left and Right Data Over Time',
    xaxis_title='Date',
    yaxis_title='Value',
    legend_title='Data Series',    
    hovermode='x unified',
    
    width=1000,
    height=600,
)

# Show plot
fig.show()


In [ ]:
idx = np.where(mp_date == np.datetime64("2011-11-29"))[0][0]
left_nns = []
right_nns = []
for nn in range(1,11,1):
    _, left_idxs = libimp.left_BIMP_kNN_double(ts_sst, window_size, period_starts, interval_length, exclude, nn, exclude_diag)
    _, right_idxs = libimp.right_BIMP_kNN_double(ts_sst, window_size, period_starts, interval_length, exclude, nn, exclude_diag)

    left_nns.append(left_idxs[idx])
    right_nns.append(right_idxs[idx])

In [ ]:
fig = go.Figure()
for year in range(start_year, end_year+1):
    if year != end_year:
        dates = np.where((mp_date >= np.datetime64(f"{year}-10-30")) & (mp_date <= np.datetime64(f"{year+1}-01-10")))[0]
    else:
        dates = np.where((mp_date >= np.datetime64(f"{year}-10-30")) & (mp_date <= np.datetime64(f"{year}-12-31")))[0]
    x = np.arange(0, len(dates), 1)
    fig.add_trace(go.Scatter(x=x, y=ts_sst[dates], mode='lines', line=dict(color="lightgrey", width=0.8), name=f'{year}',showlegend=False))

b = True
# Print the 10 left NNs
for idx in left_nns:
    dates = mp_date[idx:idx+window_size]
    year = pd.to_datetime(dates[0]).year
    shift = (dates[0] - np.datetime64(f"{year}-10-30")).astype(int)
    x1 = np.arange(window_size) + shift
    y1 = ts_sst[idx:idx+window_size]
    fig.add_trace(go.Scatter(x=x1, y=y1, mode='lines', line=dict(color=px.colors.qualitative.Vivid[7], width=2.5), showlegend=b, name ="Left NNs"))
    b = False
# Print the 10 right NNs
b = True
for idx in right_nns:
    dates = mp_date[idx:idx+window_size]
    year = pd.to_datetime(dates[0]).year
    shift = (dates[0] - np.datetime64(f"{year}-10-30")).astype(int)
    x1 = np.arange(window_size) + shift
    y1 = ts_sst[idx:idx+window_size]
    fig.add_trace(go.Scatter(x=x1, y=y1, mode='lines', line=dict(color=px.colors.qualitative.G10[8], width=2.5), showlegend=b, name ="Right NNs"))
    b = False
    
# Print the right-discord
dates = np.where((mp_date >= np.datetime64("2011-11-29")) & (mp_date <= np.datetime64("2011-12-12")))[0]
shift = (np.datetime64("2001-11-29") - np.datetime64("2001-10-30")).astype(int)
x1 = np.arange(window_size) + shift
y1 = ts_sst[dates]
fig.add_trace(go.Scatter(x=x1, y=y1, mode='lines', line=dict(color="black", width=4), name='Right Discord'))


fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    template="plotly_white",
    font_size=20,   
    showlegend=True,
    font=dict(
        family="Latin Modern Roman",
        size=20,  
        color="Black"),)
fig.update_annotations(font=dict(
        family="Latin Modern Roman",
        size=20,  
        color="Black"))
fig.update_layout(
    
    yaxis_title='Temperature (°C)',
    xaxis=dict(
        showgrid=False),
    yaxis=dict(
        tickvals=[19.85, 20, 20.5], 
        ticktext=[19.85, 20, 20.5],
        range=[19.85,20.5],
        showgrid=True))
fig.update_layout(legend=dict(    
orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="center",
    x=0.5))

dates = mp_date[np.where((mp_date >= np.datetime64("2000-10-30")) & (mp_date <= np.datetime64("2001-01-10")))[0]][1:]
formatted_dates = pd.to_datetime(dates).strftime("%d %b")

fig.update_xaxes(
    tickvals=list(range(0,len(formatted_dates), 15)),
    ticktext=formatted_dates[::15],
    showgrid=False,
    tickangle=-45)
fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)
fig.write_image("pattern_gone.pdf", engine="kaleido")
fig.show()

In [ ]:

fig = go.Figure()



# Left 10 trace with hover text
fig.add_trace(go.Scatter(
    x=mp_date[k*365:],         # Date array for x-axis
    y=left_imp_10[k*365:],     # Left 10 data
    mode='lines',
    name='c10NN left IMP',
    line=dict(color=px.colors.qualitative.Vivid[7]),
    text=hover_text_left_10,          # Custom hover text for Left 10
))

# Right trace with hover text
fig.add_trace(go.Scatter(
    x=mp_date[k*365:],         # Date array for x-axis
    y=right_imp[k*365:],       # Right data
    mode='lines',
    name='Right IMP',
    line=dict(color=px.colors.qualitative.G10[8]),
    text=hover_text_right,            # Custom hover text for Right
))



idx = np.where(mp_date == np.datetime64("2011-11-29"))[0][0]
fig.add_trace(go.Scatter(
    x=[mp_date[idx]],         
    y=[right_imp[idx]],   
    mode='markers',
    name='Right discord', 
    marker=dict(
        color='black',
        size=10
    )
))


# Customize layout
fig.update_xaxes(range=[np.datetime64("2000-01-01"), np.datetime64("2023-12-31")])
fig.update_yaxes(range=[0,0.8], 
        tickvals=[0.4, 0.8], 
        ticktext=[0.4, 0.8],
)
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Distance',
    width=800,
    height=450,
    template="plotly_white",
    font_size=20,   
    showlegend=True,
    font=dict(
        family="Latin Modern Roman",
        size=20,  
        color="Black"),)


fig.update_layout(legend=dict(    
orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="center",
    x=0.5))
    
fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)
# Show plot
fig.show()

fig.write_image("left_vs_right_IMP.pdf", engine="kaleido")
